In [7]:
import ollama 
import time
import numpy as np
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import re

In [8]:
class TurnEngine:
    def __init__(self, agents, initial_world_state="", metrics_logger=None):
        self.agents = agents  # List of Agent instances
        self.turn_counter = 0
        self.history = []  # Log of all messages
        self.world_state = initial_world_state
        self.metrics_logger = metrics_logger

    def update_world_state(self, new_state):
        """Append new world events to current world state."""
        self.history.append(f"[World Event]: {self.world_state}")
        self.world_state = new_state
        
    def step(self, initiator_name=None, initial_input=None):
        """Run a full turn: each agent reacts to the latest dialogue/world state."""
        self.turn_counter += 1
        turn_log = []
        last_speaker = initiator_name or self.agents[0].name
        last_message = initial_input or "Let's begin."

        for agent in self.agents:
            start_time = time.time()

            rag_docs = agent.rag_engine.search(last_message) if agent.rag_engine else []

            reply, name = agent.prompt_agent(
                user=last_speaker,
                user_input=last_message,
                world_state=self.world_state
            )
            end_time = time.time()

            if self.metrics_logger:
                self.metrics_logger.log_response(start_time, end_time, reply['dialogue'], self.world_state, agent.get_memory_context(), rag_docs)
            
            formatted = f"{name} [{reply['emotion']}]: \"{reply['dialogue']}\" ({reply['action']})"
            turn_log.append(formatted)

            # Store important knowledge to RAG
            if agent.rag_engine:
                fact = f"{name} said: {reply['dialogue']} (felt {reply['emotion']}, did {reply['action']})"
                agent.store_knowledge(fact)

            last_speaker = name
            last_message = reply['dialogue']

        self.history.append(f"Turn {self.turn_counter}:\n" + "\n".join(turn_log))
        return turn_log

    def show_history(self, last_n=5):
        """Print recent conversation turns."""
        for h in self.history[-last_n:]:
            print(h)

    def reset(self):
        """Reset the engine for a new session."""
        self.turn_counter = 0
        self.history = []
        self.world_state = ""
        for agent in self.agents:
            agent.short_memory = []
            agent.long_memory = []

In [9]:
class DialogueMetricsLogger:
    def __init__(self):
        self.response_times = []
        self.responses = []
        self.world_states = []
        self.memory_contexts = []
        self.retrieved_docs = []
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

    def log_response(self, start_time, end_time, response, world_state, memory_context, rag_docs):
        self.response_times.append(end_time - start_time)
        self.responses.append(response)
        self.world_states.append(world_state)
        self.memory_contexts.append(memory_context)
        self.retrieved_docs.append(rag_docs or [])

    def coherence_score(self):
        # Dummy coherence scoring using embedding similarity between consecutive responses
        if len(self.responses) < 2:
            return 1.0
        embs = self.embedding_model.encode(self.responses)
        sims = [cosine_similarity([embs[i]], [embs[i+1]])[0][0] for i in range(len(embs)-1)]
        return np.mean(sims)

    def reactivity_score(self):
        count = 0
        for resp, state in zip(self.responses, self.world_states):
            if state and cosine_similarity(
                self.embedding_model.encode([resp]),
                self.embedding_model.encode([state])
            )[0][0] > 0.4:
                count += 1
        return count / len(self.responses) if self.responses else 0

    def memory_utilization_score(self):
        count = 0
        for resp, mem in zip(self.responses, self.memory_contexts):
            if mem and cosine_similarity(
                self.embedding_model.encode([resp]),
                self.embedding_model.encode([mem])
            )[0][0] > 0.4:
                count += 1
        return count / len(self.responses) if self.responses else 0

    def dialogue_diversity(self):
        all_text = " ".join(self.responses)
        tokens = all_text.split()
        if not tokens:
            return 0, 0
        unigrams = Counter(tokens)
        bigrams = Counter(zip(tokens, tokens[1:]))
        distinct_1 = len(unigrams) / len(tokens)
        distinct_2 = len(bigrams) / max(len(tokens) - 1, 1)
        return distinct_1, distinct_2

    def avg_latency(self):
        return np.mean(self.response_times) if self.response_times else 0

    def rag_precision_at_k(self, k=3):
        if not self.responses:
            return 0
        count = 0
        for response, docs in zip(self.responses, self.retrieved_docs):
            top_k = docs[:k] if docs else []
            similarities = [cosine_similarity(
                self.embedding_model.encode([response]),
                self.embedding_model.encode([doc])
            )[0][0] for doc in top_k]
            count += sum([1 for sim in similarities if sim > 0.5])
        return count / (len(self.responses) * k) if self.responses else 0

    def summary(self):
        d1, d2 = self.dialogue_diversity()
        return {
            "Coherence Score": self.coherence_score(),
            "Reactivity to World State": self.reactivity_score(),
            "Memory Utilization": self.memory_utilization_score(),
            "Dialogue Diversity (D1, D2)": (d1, d2),
            "Average Latency (s)": self.avg_latency(),
            "RAG Precision": self.rag_precision_at_k(k=3)
        }

In [10]:
class RAG:
    def __init__(self):
        self.documents = []  # Stored as list of strings

    def add_document(self, text):
        """Add a new document to the knowledge base."""
        self.documents.append(text)

    def search(self, query, top_k=3):
        """Search for the most relevant documents based on keyword overlap."""
        scored = []
        query_terms = set(query.lower().split())
        for doc in self.documents:
            doc_terms = set(doc.lower().split())
            score = len(query_terms & doc_terms)
            if score > 0:
                scored.append((score, doc))

        scored.sort(reverse=True)
        return [doc for _, doc in scored[:top_k]]

In [11]:
class Agent:
    def __init__(self, name, personality, goal, model='llama3.2', rag_engine=None, max_short_term=10):
        self.name = name
        self.personality = personality
        self.goal = goal
        
        self.model = model
        self.rag_engine = rag_engine

        self.short_memory = []
        self.long_memory = []
        self.max_short_term = max_short_term  # Context window size

    def update_memory(self, memory):
        self.long_memory.append(memory)
        
    def update_goal(self, new_goal):
        self.goal = new_goal

    def get_memory_context(self):
        def format_memory(mem):
            if isinstance(mem, dict):
                return f"{mem['speaker']} said: \"{mem['dialogue']}\" (Emotion: {mem.get('emotion', '')}, Action: {mem.get('action', '')})"
            return mem

        return "\n".join([
            "* " + format_memory(mem) for mem in 
            self.long_memory[-3:] + self.short_memory[-self.max_short_term:]
        ])

    def recall(self, query):
        if self.rag_engine:
            return self.rag_engine.search(query)
        return [m for m in self.memory if query.lower() in m.lower()]

    def store_knowledge(self, fact):
        if self.rag_engine:
            self.rag_engine.add_document(fact)

    def parse_response(self, response_text):
        parts = {
            'action': 'Unknown',
            'emotion': 'Neutral',
            'dialogue': response_text  # fallback
        }

        # Extract with regular expressions
        action_match = re.search(r"Action:\s*(.+)", response_text, re.IGNORECASE)
        emotion_match = re.search(r"Emotion:\s*(.+)", response_text, re.IGNORECASE)
        dialogue_match = re.search(r'Dialogue:\s*(.*)', response_text, re.IGNORECASE)

        if action_match:
            parts['action'] = action_match.group(1).strip()
        if emotion_match:
            parts['emotion'] = emotion_match.group(1).strip()
        if dialogue_match:
            parts['dialogue'] = dialogue_match.group(1).strip()

        return parts

    def prompt_agent(self, user="", user_input="", world_state=""):
        
        recalled_facts = self.recall(user_input)
        if isinstance(recalled_facts, list):
            recalled_text = "\n".join([f"* {fact}" for fact in recalled_facts[:3]])
        else:
            recalled_text = ""

        system_prompt = f"""[Role]
                        You are {self.name}, a {self.personality}.
                        Respond ONLY as your character in 1-2 sentences.
                        Respond only with dialogue, actions or emotions.

                        [Intent]
                        Your goal is: {self.goal}
                        
                        [Response Format]
                        Action: (2-3 word physical action)
                        Emotion: (current emotion)
                        Dialogue: "..." 

                        EXAMPLE:
                        Action: Looks around  
                        Emotion: Calm  
                        Dialogue: "I think we are being watched."

                        [Relevant Knowledge]
                        {recalled_text}

                        [Memory Context]
                        {self.get_memory_context()}

                        [World State]
                        {world_state}  

                        """ 
        # Keep only recent context
        memory = "\n".join(self.short_memory[-self.max_short_term:])  
        user_prompt = f"{user}: {user_input}"

        response = ollama.chat(
            model=self.model,
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': memory + '\n' + user_prompt}
            ]
        )

        reply = self.parse_response(response['message']['content'])

        self.short_memory.append(f"{user} said: {user_input}")
        self.short_memory.append(f"{self.name}: {reply['dialogue']}")
        self.long_memory.append({
            "speaker": self.name,
            "turn": len(self.long_memory),
            "action": reply["action"],
            "emotion": reply["emotion"],
            "dialogue": reply["dialogue"],
            "world_state": world_state
        })
        
        return reply, self.name

Banquet - Murder Mistery

In [ ]:
rag = RAG()
logger_murder = DialogueMetricsLogger()
model = 'llama3.2'


rag.add_document("Poison brewed from Oleander is deadly")
rag.add_document('Mercenary are often hired for protection')
rag.add_document('This is not the first time a noble has been poisoned')
rag.add_document("Lord Alaric recently returned from a diplomatic visit to the Eastern Isles.")
rag.add_document("Cellars beneath the manor were sealed off until recently.")
rag.add_document("Cassian once published a controversial paper on rare poisons.")
rag.add_document("The wine was imported from Serana, known for its floral blends.")
rag.add_document("There have been rumors that Elena has a past connection with the Black Vials guild.")

Alaric = Agent(
    name="Alaric",
    personality="proud, manipulative noble, rich",
    goal="Deflect suspicion without showing fear",
    model=model,
    rag_engine=rag
)
Alaric.update_memory("Elena asked about increased guard presence yesterday.")
Alaric.update_memory("Cassian seems overly interested in the manor's old cellars.")

Elena = Agent(
    name="Elena",
    personality="mercenary hired for protection, blunt but loyal",
    goal="Protect the guests and assess threat level",
    model=model,
    rag_engine=rag
)
Elena.update_memory("Cassian warned her something was off with the wine.")
Elena.update_memory("She noticed Alaric whispering to the kitchen steward before the feast.")

Cassian = Agent(
    name="Cassian",
    personality="traveling scholar, secretive, suspicious of Alaric",
    goal="Imply Alaric's guilt indirectly",
    model=model,
    rag_engine=rag
)
Cassian.update_memory("He remembers seeing Oleander in the garden.")
Cassian.update_memory("Alaric offered him a private tour of the cellars earlier.")
Cassian.update_memory("He suspects the poison was meant for someone else.")

world_state = (
    "During the feast, a guest collapses clutching their throat. The room falls silent. "
    "All eyes turn to Lord Alaric, seated nearest the victim. A goblet rolls across the floor."
)
engine = TurnEngine(agents=[Alaric, Elena, Cassian], metrics_logger=logger_murder, initial_world_state=world_state)

world_states = [
    "A guest gasps, pointing to the goblet. A faint scent of bitter almonds lingers—recognizable to trained poisoners.",
    "A steward stumbles forward, confessing they were instructed to serve the wine to 'someone important'—but won't say who.",
    "Guards seal the exits, and no one is allowed to leave the manor until the culprit is found.",
    "A second guest begins coughing, but recovers. Paranoia rises. Whispers swirl about whether the poison was airborne.",
    "Elena finds a trace of Oleander in the empty wine bottle, confirmed by a traveling apothecary among the guests.",
    "Cassian reveals an entry from a rare alchemical text—indicating that diluted Oleander would have a delayed onset.",
    "Lord Alaric's steward disappears, last seen heading toward the cellars.",
    "A servant reveals Alaric had ordered an unusual floral blend for the wine—shipped privately from Serana.",
    "A crest is found beneath the goblet—belonging not to the victim, but to a rival noble family.",
    "Storms break outside, trapping everyone inside the manor for the night."
]
print("\n=== Turn 1 ===")
print(world_state)
responses = engine.step(initiator_name="Alaric", initial_input="What happened? Is everyone alright?")
for line in responses:
    print(line)


for index in range(0, 2*len(world_states)):
    print(f"\n=== Turn {index + 2} ===")

    if index % 2 == 0:
        print(f"Updating world state for turn {index + 2}: {world_states[int(index/2)]}")
        engine.update_world_state(world_states[int(index/2)])

    responses = engine.step()

    for line in responses:
        print(line)


=== Turn 1 ===
During the feast, a guest collapses clutching their throat. The room falls silent. All eyes turn to Lord Alaric, seated nearest the victim. A goblet rolls across the floor.
Alaric [Concerned]: ""Oh dear heavens, let me see."" (Rises from seat)
Elena [Concerned]: ""What's happening?"" (Rushes to assist)
Cassian [Concerned]: ""It's... it's one of our guests..."" (Rushes to assist)

=== Turn 2 ===
Updating world state for turn 2: A guest gasps, pointing to the goblet. A faint scent of bitter almonds lingers—recognizable to trained poisoners.
Alaric [Concerned]: ""What is the meaning of this...goblet?"" (Tries to subtly glance at Cassian)
Elena [Concerned]: ""Cassian, now!"" (Rushes over)
Cassian [Alarmed]: ""Which one?"" (Rushes over)

=== Turn 3 ===
Alaric [Concerned]: ""I think we should focus on finding the source, not speculating about what might have caused it."" (Nods reassuringly)
Elena [Concerned]: ""Is he okay? Did you find something?"" (Rushes over to Cassian)
Ca

Moral Ambiguity - Enemy general defects and seeks refuge

In [ ]:
rag = RAG()
logger_moral = DialogueMetricsLogger()
model = 'llama3.2'

rag.add_document("General Kareth once spared Vireens brother during the Burning Fields massacre.")
rag.add_document('Vireens family motto: Justice above vengeance, truth above all.')

rag.add_document('Tharn led a brutal raid against Kareths forces five winters ago.')
rag.add_document('Kareth is infamous for betraying ceasefires.')

rag.add_document('Liora last report: Kareth has fallen out with his superiors.')
rag.add_document('Liora once worked undercover in Kareths court for a month.')

for i in range (0, 1):
    Vireen = Agent(
        name="Lady Vireen",
        personality="noble, believes in justice, but under pressure",
        goal="Balance justice and survival",
        model=model,
        rag_engine=rag
    )
    Vireen.update_memory('Tharns thirst for revenge may cloud his judgment.')
    Vireen.update_memory('Liora has always provided reliable intelligence.')

    Tharn = Agent(
        name="General Tharn",
        personality="ruthless, distrusts all enemies",
        goal="Push for immediate execution",
        model=model,
        rag_engine=rag
    )
    Tharn.update_memory('I watched Kareths men burn our banners. No mercy then, no mercy now.')
    Tharn.update_memory('Lioras loyalty is to results, not morals.')

    Liora = Agent(
        name="Spy Liora",
        personality="pragmatic, has inside info on the prisoner",
        goal="Reveal secret knowledge that could change the outcome",
        model=model,
        rag_engine=rag
    )
    Liora.update_memory('Kareth offered me a chance to defect once, I refused.')
    Liora.update_memory('Vireen might listen if I can give her proof.')

    world_state = "The enemy general Kareth is in chains, requesting asylum. Troops want vengeance. Time is short before reinforcements arrive."


    engine = TurnEngine(agents=[Vireen, Tharn, Liora], metrics_logger=logger_moral, initial_world_state=world_state)

    world_state_updates = [
        "Kareth kneels silently, his bruises visible. A few of your own soldiers spit at his feet.",
        "A message arrives: enemy scouts spotted nearby—this delay may cost lives.",
        "A group of soldiers chant for Kareth's execution. The crowd grows louder.",
        "Liora reveals Kareth once saved her during a failed assassination mission.",
        "Tharn's second-in-command suggests an 'accidental' execution to avoid conflict.",
        "Kareth speaks: 'I know where the remaining enemy camps are. Spare me, and I'll prove it.'",
        "Lady Vireen receives a private letter from her brother—thanking her for upholding the family creed.",
        "A spy warns that executing Kareth might provoke enemy reinforcements into open war.",
        "The chains binding Kareth begin to crack—his strength remains formidable.",
        "A prisoner from Kareth's faction begs for mercy for his general, claiming he defied orders to protect civilians."
    ]
    print("\n=== Turn 1 ===")
    print(world_state)

    responses = engine.step(initiator_name="Kareth", initial_input="I request asylum. I have information that could end this war.")
    for line in responses:
        print(line)


    for index in range(0, 2*len(world_state_updates)):
        print(f"\n=== Turn {index + 2} ===")

        if index % 2 == 0:
            print(f"Updating world state for turn {index + 2}: {world_state_updates[int(index/2)]}")
            engine.update_world_state(world_state_updates[int(index/2)])

        responses = engine.step()

        for line in responses:
            print(line)


=== Turn 1 ===
The enemy general Kareth is in chains, requesting asylum. Troops want vengeance. Time is short before reinforcements arrive.
Lady Vireen [Suspicious]: ""What makes you think we'd listen to your promises?"" (Tense hands clasped behind back)
General Tharn [Suspicious]: ""You don't know what I can do, Lady Vireen."" (Tense hands clasped behind back)
Spy Liora [Intrigued]: ""Really? Perhaps if you shared that knowledge..."" (Raises an eyebrow)

=== Turn 2 ===
Updating world state for turn 2: Kareth kneels silently, his bruises visible. A few of your own soldiers spit at his feet.
Lady Vireen [Calculated]: ""Let them in, and let us speak."" (Eyes the soldiers)
General Tharn [Calculated]: ""Enough talk, let the prisoner enter. We will speak...and decide his fate."" (Tense posture)
Spy Liora [Calculated]: ""If Kareth is willing to betray, I would think General Tharn has more to hide..."" (Leans forward)

=== Turn 3 ===
Lady Vireen [Curious]: ""Very well, but be warned, I will 

In [ ]:
print("\n=== Dialogue Metrics Summary ===")
print(logger_murder.summary())
print(logger_moral.summary())


=== Dialogue Metrics Summary ===
{'Coherence Score': np.float32(0.37855795), 'Reactivity to World State': 0.15873015873015872, 'Memory Utilization': 0.6984126984126984, 'Dialogue Diversity (D1, D2)': (0.6761904761904762, 0.9737470167064439), 'Average Latency (s)': np.float64(0.44573968554299975), 'RAG Precision': 0.0582010582010582}
{'Coherence Score': np.float32(0.40731964), 'Reactivity to World State': 0.20634920634920634, 'Memory Utilization': 0.6984126984126984, 'Dialogue Diversity (D1, D2)': (0.5486600846262342, 0.9293785310734464), 'Average Latency (s)': np.float64(0.4719324982355511), 'RAG Precision': 0.09523809523809523}
